In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
ratings=pd.read_csv('https://raw.githubusercontent.com/AkshatSan/MovieRecomm/main/data/ratings.csv',sep='\t')

In [3]:
users=pd.read_csv('https://raw.githubusercontent.com/AkshatSan/MovieRecomm/main/data/users.csv',encoding='cp1252',sep='\t')

In [4]:
movies = pd.read_csv('movies.csv',sep='\t',encoding='latin-1', usecols=['movie_id', 'title', 'genres'])

In [5]:
movies.head(1)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy


In [6]:
ratings.head(1)

,Unnamed: 0,user_id,movie_id,rating,timestamp,user_emb_id,movie_emb_id
0,0,1,1193,5,978300760,0,1192


In [7]:
ratings.columns

Index(['Unnamed: 0', 'user_id', 'movie_id', 'rating', 'timestamp',
       'user_emb_id', 'movie_emb_id'],
      dtype='object')

In [8]:
users.head(1)

,Unnamed: 0,user_id,gender,age,occupation,zipcode,age_desc,occ_desc
0,0,1,F,1,10,48067,Under 18,K-12 student


In [9]:
users.drop(columns='Unnamed: 0',axis=1,inplace=True)
ratings.drop(columns='Unnamed: 0',axis=1,inplace=True)

In [10]:
user_rating=users.merge(ratings,how='inner',on='user_id')

In [11]:
user_rating.head(1)

,user_id,gender,age,occupation,zipcode,age_desc,occ_desc,movie_id,rating,timestamp,user_emb_id,movie_emb_id
0,1,F,1,10,48067,Under 18,K-12 student,1193,5,978300760,0,1192


In [12]:
df=user_rating.merge(movies,how='inner',on='movie_id')

In [13]:
df.head()

,user_id,gender,age,occupation,zipcode,age_desc,occ_desc,movie_id,rating,timestamp,user_emb_id,movie_emb_id,title,genres
0,1,F,1,10,48067,Under 18,K-12 student,1193,5,978300760,0,1192,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,M,56,16,70072,56+,self-employed,1193,5,978298413,1,1192,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,M,25,12,32793,25-34,programmer,1193,4,978220179,11,1192,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,M,25,7,22903,25-34,executive/managerial,1193,4,978199279,14,1192,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,M,50,1,95350,50-55,academic/educator,1193,5,978158471,16,1192,One Flew Over the Cuckoo's Nest (1975),Drama


In [14]:
df.drop(columns=['user_id','timestamp','occ_desc','movie_id'],axis=1,inplace=True)

In [15]:
df.head(1)

,gender,age,occupation,zipcode,age_desc,rating,user_emb_id,movie_emb_id,title,genres
0,F,1,10,48067,Under 18,5,0,1192,One Flew Over the Cuckoo's Nest (1975),Drama


In [16]:
df.rename(columns={'user_emb_id':'user_id',
                   'movie_emb_id':'movie_id'},inplace=True)

In [17]:
df.head(1)

,gender,age,occupation,zipcode,age_desc,rating,user_id,movie_id,title,genres
0,F,1,10,48067,Under 18,5,0,1192,One Flew Over the Cuckoo's Nest (1975),Drama


In [18]:
df["genres"]

0                         Drama
1                         Drama
2                         Drama
3                         Drama
4                         Drama
                   ...         
1000204             Documentary
1000205                   Drama
1000206                   Drama
1000207    Comedy|Drama|Western
1000208             Documentary
Name: genres, Length: 1000209, dtype: object

In [19]:
df.explode('genres')

,gender,age,occupation,zipcode,age_desc,rating,user_id,movie_id,title,genres
0,F,1,10,48067,Under 18,5,0,1192,One Flew Over the Cuckoo's Nest (1975),Drama
1,M,56,16,70072,56+,5,1,1192,One Flew Over the Cuckoo's Nest (1975),Drama
2,M,25,12,32793,25-34,4,11,1192,One Flew Over the Cuckoo's Nest (1975),Drama
3,M,25,7,22903,25-34,4,14,1192,One Flew Over the Cuckoo's Nest (1975),Drama
4,M,50,1,95350,50-55,5,16,1192,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...,...,...,...,...
1000204,M,18,17,47901,18-24,5,5948,2197,Modulations (1998),Documentary
1000205,M,35,14,30030,35-44,3,5674,2702,Broken Vessels (1998),Drama
1000206,M,18,17,92886,18-24,1,5779,2844,White Boys (1999),Drama
1000207,F,18,20,55410,18-24,5,5850,3606,One Little Indian (1973),Comedy|Drama|Western


In [20]:
a=df.groupby('title')['user_id'].count()>=100
movies_to_consider=a[a]

In [21]:
movies_to_consider=list(movies_to_consider.index)

In [22]:
df.shape

(1000209, 10)

In [23]:
r=df['title'].isin(movies_to_consider)
df=df[r]

In [24]:
df

,gender,age,occupation,zipcode,age_desc,rating,user_id,movie_id,title,genres
0,F,1,10,48067,Under 18,5,0,1192,One Flew Over the Cuckoo's Nest (1975),Drama
1,M,56,16,70072,56+,5,1,1192,One Flew Over the Cuckoo's Nest (1975),Drama
2,M,25,12,32793,25-34,4,11,1192,One Flew Over the Cuckoo's Nest (1975),Drama
3,M,25,7,22903,25-34,4,14,1192,One Flew Over the Cuckoo's Nest (1975),Drama
4,M,50,1,95350,50-55,5,16,1192,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...,...,...,...,...
993730,F,35,14,38018,35-44,4,5717,1757,Dangerous Beauty (1998),Drama
993731,F,35,2,78744,35-44,4,5754,1757,Dangerous Beauty (1998),Drama
993732,F,25,7,92120,25-34,3,5811,1757,Dangerous Beauty (1998),Drama
993733,M,25,1,92120,25-34,3,5830,1757,Dangerous Beauty (1998),Drama


In [25]:
df_female=df[df["gender"]=='F']
df_male=df[df["gender"]=='M']

In [26]:
df_female.drop(columns='gender',axis=1,inplace=True)
df_male.drop(columns='gender',axis=1,inplace=True)

In [27]:
df_male.head(1)

,age,occupation,zipcode,age_desc,rating,user_id,movie_id,title,genres
1,56,16,70072,56+,5,1,1192,One Flew Over the Cuckoo's Nest (1975),Drama


In [28]:
df_female.drop(columns='zipcode',axis=1,inplace=True)
df_male.drop(columns='zipcode',axis=1,inplace=True)

In [29]:
df_male

,age,occupation,age_desc,rating,user_id,movie_id,title,genres
1,56,16,56+,5,1,1192,One Flew Over the Cuckoo's Nest (1975),Drama
2,25,12,25-34,4,11,1192,One Flew Over the Cuckoo's Nest (1975),Drama
3,25,7,25-34,4,14,1192,One Flew Over the Cuckoo's Nest (1975),Drama
4,50,1,50-55,5,16,1192,One Flew Over the Cuckoo's Nest (1975),Drama
6,1,10,Under 18,5,18,1192,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...,...,...
993722,18,0,18-24,4,5393,1757,Dangerous Beauty (1998),Drama
993723,25,12,25-34,4,5465,1757,Dangerous Beauty (1998),Drama
993724,35,1,35-44,5,5478,1757,Dangerous Beauty (1998),Drama
993729,35,14,35-44,4,5674,1757,Dangerous Beauty (1998),Drama


In [30]:
df_male["genres"].head(30)

1     Drama
2     Drama
3     Drama
4     Drama
6     Drama
9     Drama
10    Drama
11    Drama
12    Drama
13    Drama
14    Drama
15    Drama
16    Drama
17    Drama
18    Drama
21    Drama
25    Drama
28    Drama
29    Drama
30    Drama
31    Drama
32    Drama
33    Drama
34    Drama
35    Drama
38    Drama
40    Drama
41    Drama
42    Drama
44    Drama
Name: genres, dtype: object

In [31]:
df_male.info()

<class 'pandas.core.frame.DataFrame'>
Index: 711445 entries, 1 to 993733
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   age         711445 non-null  int64 
 1   occupation  711445 non-null  int64 
 2   age_desc    711445 non-null  object
 3   rating      711445 non-null  int64 
 4   user_id     711445 non-null  int64 
 5   movie_id    711445 non-null  int64 
 6   title       711445 non-null  object
 7   genres      711445 non-null  object
dtypes: int64(5), object(3)
memory usage: 48.9+ MB


In [32]:
df['occupation'].value_counts()

occupation
4     123938
0     121537
7      99721
1      80366
17     69632
20     55218
12     55062
14     46896
2      46735
16     43180
6      34960
3      29587
15     22052
10     21901
5      20675
11     19578
19     13812
13     12826
18     11210
9      10798
8       2541
Name: count, dtype: int64

In [33]:
df_male['age_desc'].value_counts().index

Index(['25-34', '35-44', '18-24', '45-49', '50-55', '56+', 'Under 18'], dtype='object', name='age_desc')

In [34]:
df_female['age_desc'].value_counts().index

Index(['25-34', '35-44', '18-24', '45-49', '50-55', '56+', 'Under 18'], dtype='object', name='age_desc')

In [35]:
df_female['age_desc'].unique()

array(['Under 18', '18-24', '25-34', '50-55', '35-44', '45-49', '56+'],
      dtype=object)

In [36]:
d={}
c=1
for i in df_female['age_desc'].unique():
    d[i]=c
    c+=1

In [122]:
new_df=pd.DataFrame()

In [123]:
new_df=df

In [125]:
type(new_df)

pandas.core.frame.DataFrame

In [37]:
d

{'Under 18': 1,
 '18-24': 2,
 '25-34': 3,
 '50-55': 4,
 '35-44': 5,
 '45-49': 6,
 '56+': 7}

In [38]:
df_female['age_desc']=df_female['age_desc'].map(d)
df_male['age_desc']=df_male['age_desc'].map(d)

In [39]:
df_female.drop(columns='age',axis=1,inplace=True)
df_male.drop(columns='age',axis=1,inplace=True)

In [40]:
df_female.drop(columns='occupation',axis=1,inplace=True)
df_male.drop(columns='occupation',axis=1,inplace=True)

**STREAMLIT**

In [41]:
import streamlit as st

In [42]:
st.header('Movie Recommendation system')
st.subheader('User need to input following information to get 5 recommended movies to watch')
st.selectbox('Gender',df["gender"].unique())
st.selectbox('enter age bracket',df['age_desc'].unique())
st.selectbox('user_id',df['user_id'].unique())

2023-11-16 17:01:24.404 
  command:

    streamlit run c:\Users\A5230\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


0

In [43]:
df_female

,age_desc,rating,user_id,movie_id,title,genres
0,1,5,0,1192,One Flew Over the Cuckoo's Nest (1975),Drama
5,2,4,17,1192,One Flew Over the Cuckoo's Nest (1975),Drama
7,3,5,23,1192,One Flew Over the Cuckoo's Nest (1975),Drama
8,3,3,27,1192,One Flew Over the Cuckoo's Nest (1975),Drama
19,4,4,58,1192,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
993728,2,3,5648,1757,Dangerous Beauty (1998),Drama
993730,5,4,5717,1757,Dangerous Beauty (1998),Drama
993731,5,4,5754,1757,Dangerous Beauty (1998),Drama
993732,3,3,5811,1757,Dangerous Beauty (1998),Drama


In [44]:
df_female['genres']

0         Drama
5         Drama
7         Drama
8         Drama
19        Drama
          ...  
993728    Drama
993730    Drama
993731    Drama
993732    Drama
993734    Drama
Name: genres, Length: 230780, dtype: object

In [45]:
df_female.info()

<class 'pandas.core.frame.DataFrame'>
Index: 230780 entries, 0 to 993734
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   age_desc  230780 non-null  int64 
 1   rating    230780 non-null  int64 
 2   user_id   230780 non-null  int64 
 3   movie_id  230780 non-null  int64 
 4   title     230780 non-null  object
 5   genres    230780 non-null  object
dtypes: int64(4), object(2)
memory usage: 12.3+ MB


In [46]:
def convert_into_list(obj):
    l=[]
    s= str(obj)
    if s.count('|')==0:
        l.append(s)
    else:
        l.append(s.split('|'))
        l=[i for t in l for i in t]
    return l        

In [47]:
df_female['genres']=df_female['genres'].apply(convert_into_list)
df_male['genres']=df_male['genres'].apply(convert_into_list)

In [48]:
df_female.head()

,age_desc,rating,user_id,movie_id,title,genres
0,1,5,0,1192,One Flew Over the Cuckoo's Nest (1975),[Drama]
5,2,4,17,1192,One Flew Over the Cuckoo's Nest (1975),[Drama]
7,3,5,23,1192,One Flew Over the Cuckoo's Nest (1975),[Drama]
8,3,3,27,1192,One Flew Over the Cuckoo's Nest (1975),[Drama]
19,4,4,58,1192,One Flew Over the Cuckoo's Nest (1975),[Drama]


In [49]:
df_female=df_female.explode('genres')
df_male=df_male.explode('genres')

In [50]:
df_male.head()

,age_desc,rating,user_id,movie_id,title,genres
1,7,5,1,1192,One Flew Over the Cuckoo's Nest (1975),Drama
2,3,4,11,1192,One Flew Over the Cuckoo's Nest (1975),Drama
3,3,4,14,1192,One Flew Over the Cuckoo's Nest (1975),Drama
4,4,5,16,1192,One Flew Over the Cuckoo's Nest (1975),Drama
6,1,5,18,1192,One Flew Over the Cuckoo's Nest (1975),Drama


In [51]:
df_female.shape

(482858, 6)

In [52]:
df_female['genres'].unique()

array(['Drama', 'Animation', "Children's", 'Musical', 'Romance', 'Comedy',
       'Action', 'Adventure', 'Fantasy', 'Sci-Fi', 'War', 'Thriller',
       'Crime', 'Mystery', 'Western', 'Horror', 'Film-Noir',
       'Documentary'], dtype=object)

In [53]:
df_female['genres'].value_counts().sort_values(ascending=False)

genres
Comedy         91637
Drama          90153
Romance        47587
Action         45125
Thriller       39044
Sci-Fi         27080
Adventure      26517
Children's     20051
Crime          15958
War            13744
Horror         13698
Musical        13144
Animation      11968
Mystery         9754
Fantasy         8542
Film-Noir       4111
Western         3337
Documentary     1408
Name: count, dtype: int64

In [54]:
df_female.head()

,age_desc,rating,user_id,movie_id,title,genres
0,1,5,0,1192,One Flew Over the Cuckoo's Nest (1975),Drama
5,2,4,17,1192,One Flew Over the Cuckoo's Nest (1975),Drama
7,3,5,23,1192,One Flew Over the Cuckoo's Nest (1975),Drama
8,3,3,27,1192,One Flew Over the Cuckoo's Nest (1975),Drama
19,4,4,58,1192,One Flew Over the Cuckoo's Nest (1975),Drama


In [55]:
df_female['age_desc'].unique()

array([1, 2, 3, 4, 5, 6, 7], dtype=int64)

In [56]:
n_d={}
for i,j in list(zip(df_male['genres'].value_counts().sort_values(ascending=False).index,np.linspace(5,1,18))):
    n_d[i]=j

In [57]:
df_male['genres']=df_male['genres'].map(n_d)

In [58]:
n_d_f={}
for i,j in list(zip(df_female['genres'].value_counts().sort_values(ascending=False).index,np.linspace(5,1,18))):
    n_d_f[i]=j

In [ ]:
df_female['age_desc'].unique()

In [59]:
df_female['genres']=df_female['genres'].map(n_d_f)

In [60]:
df_female['weighted_average']=(2*df_female['rating']+df_female['genres'])/3
df_male['weighted_average']=(2*df_male['rating']+df_male['genres'])/3

In [76]:
df_female.info()

<class 'pandas.core.frame.DataFrame'>
Index: 482858 entries, 0 to 993734
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   age_desc          482858 non-null  int64  
 1   rating            482858 non-null  int64  
 2   user_id           482858 non-null  int64  
 3   movie_id          482858 non-null  int64  
 4   title             482858 non-null  object 
 5   genres            482858 non-null  float64
 6   weighted_average  482858 non-null  float64
dtypes: float64(2), int64(4), object(1)
memory usage: 29.5+ MB


In [61]:
f_age_1=df_female[df_female['age_desc']==1]
f_age_2=df_female[df_female['age_desc']==2]
f_age_3=df_female[df_female['age_desc']==3]
f_age_4=df_female[df_female['age_desc']==4]
f_age_5=df_female[df_female['age_desc']==5]
f_age_6=df_female[df_female['age_desc']==6]
f_age_7=df_female[df_female['age_desc']==7]
m_age_1=df_male[df_male['age_desc']==1]
m_age_2=df_male[df_male['age_desc']==2]
m_age_3=df_male[df_male['age_desc']==3]
m_age_4=df_male[df_male['age_desc']==4]
m_age_5=df_male[df_male['age_desc']==5]
m_age_6=df_male[df_male['age_desc']==6]
m_age_7=df_male[df_male['age_desc']==7]

In [62]:
f_age_1

,age_desc,rating,user_id,movie_id,title,genres,weighted_average
0,1,5,0,1192,One Flew Over the Cuckoo's Nest (1975),4.764706,4.921569
27,1,2,98,1192,One Flew Over the Cuckoo's Nest (1975),4.764706,2.921569
208,1,5,742,1192,One Flew Over the Cuckoo's Nest (1975),4.764706,4.921569
303,1,5,1087,1192,One Flew Over the Cuckoo's Nest (1975),4.764706,4.921569
385,1,4,1364,1192,One Flew Over the Cuckoo's Nest (1975),4.764706,4.254902
...,...,...,...,...,...,...,...
990846,1,1,4574,1331,"Believers, The (1987)",2.647059,1.549020
990846,1,1,4574,1331,"Believers, The (1987)",4.058824,2.019608
991578,1,1,3842,3444,Eyes of Laura Mars (1978),1.941176,1.313725
991578,1,1,3842,3444,Eyes of Laura Mars (1978),4.058824,2.019608


In [77]:
def drop(data,col):
    return data.drop(columns=col,axis=1,inplace=True)


In [80]:
f_age_7

,rating,user_id,movie_id,title,genres,weighted_average
24,5,88,1192,One Flew Over the Cuckoo's Nest (1975),4.764706,4.921569
88,3,313,1192,One Flew Over the Cuckoo's Nest (1975),4.764706,3.588235
89,5,314,1192,One Flew Over the Cuckoo's Nest (1975),4.764706,4.921569
91,4,317,1192,One Flew Over the Cuckoo's Nest (1975),4.764706,4.254902
99,5,333,1192,One Flew Over the Cuckoo's Nest (1975),4.764706,4.921569
...,...,...,...,...,...,...
990715,5,5985,1361,"Garden of Finzi-Contini, The (Giardino dei Fin...",4.764706,4.921569
991618,2,4887,3444,Eyes of Laura Mars (1978),1.941176,1.980392
991618,2,4887,3444,Eyes of Laura Mars (1978),4.058824,2.686275
991625,4,5220,3444,Eyes of Laura Mars (1978),1.941176,3.313725


In [78]:
drop(f_age_1,'age_desc')
drop(f_age_2,'age_desc')
drop(f_age_3,'age_desc')
drop(f_age_4,'age_desc')
drop(f_age_5,'age_desc')
drop(f_age_6,'age_desc')
drop(f_age_7,'age_desc')

KeyError: "['age_desc'] not found in axis"

In [79]:
drop(m_age_1,'age_desc')
drop(m_age_2,'age_desc')
drop(m_age_3,'age_desc')
drop(m_age_4,'age_desc')
drop(m_age_5,'age_desc')
drop(m_age_6,'age_desc')
drop(m_age_7,'age_desc')

KeyError: "['age_desc'] not found in axis"

In [66]:
m_age_1

,rating,user_id,movie_id,title,genres,weighted_average
6,5,18,1192,One Flew Over the Cuckoo's Nest (1975),4.764706,4.921569
103,4,348,1192,One Flew Over the Cuckoo's Nest (1975),4.764706,4.254902
203,4,718,1192,One Flew Over the Cuckoo's Nest (1975),4.764706,4.254902
234,4,828,1192,One Flew Over the Cuckoo's Nest (1975),4.764706,4.254902
268,4,959,1192,One Flew Over the Cuckoo's Nest (1975),4.764706,4.254902
...,...,...,...,...,...,...
989581,1,5557,2094,"Shaggy D.A., The (1976)",2.647059,1.549020
989581,1,5557,2094,"Shaggy D.A., The (1976)",5.000000,2.333333
990704,5,5254,1361,"Garden of Finzi-Contini, The (Giardino dei Fin...",4.764706,4.921569
991626,4,5254,3444,Eyes of Laura Mars (1978),2.176471,3.392157


In [67]:
a1=m_age_1.pivot_table(values='weighted_average',columns='title',index='user_id').fillna(0)

In [68]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(a1)

In [69]:
similarity

array([[1.        , 0.08844142, 0.42613796, ..., 0.18186673, 0.28756163,
        0.22488084],
       [0.08844142, 1.        , 0.11460495, ..., 0.03628654, 0.08282495,
        0.04409293],
       [0.42613796, 0.11460495, 1.        , ..., 0.17705675, 0.18676613,
        0.23654156],
       ...,
       [0.18186673, 0.03628654, 0.17705675, ..., 1.        , 0.22557012,
        0.06714825],
       [0.28756163, 0.08282495, 0.18676613, ..., 0.22557012, 1.        ,
        0.27852489],
       [0.22488084, 0.04409293, 0.23654156, ..., 0.06714825, 0.27852489,
        1.        ]])

In [70]:
a1

title,"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",2 Days in the Valley (1996),20 Dates (1998),"20,000 Leagues Under the Sea (1954)",...,Yellow Submarine (1968),Yojimbo (1961),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Your Friends and Neighbors (1998),Zero Effect (1998),eXistenZ (1999)
user_id,,,,,,,,,,,,,,,,,,,,,
18,0.0,0.0,0.000000,3.509804,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,4.686275,0.0,0.0,3.169935,0.0,0.000000,2.098039
152,0.0,0.0,4.764706,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
348,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
489,0.0,0.0,0.000000,4.176471,3.27451,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
579,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5756,0.0,0.0,0.000000,0.000000,0.00000,0.000000,2.660131,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5776,0.0,0.0,0.000000,0.000000,0.00000,4.921569,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5802,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000


In [71]:
a1.columns[1]

'...And Justice for All (1979)'

In [72]:
def recommend(user_id,gender,age_desc):
    #gender=input("male or female: ")
    #age_desc=int(input("Under 18': 1,'18-24': 2, '25-34': 3,'50-55': 4,'35-44': 5,'45-49': 6,'56+': 7"))
    l=[]
    index=np.where(a1.index==user_id)[0][0]
    if gender=='M':
        if age_desc==1:
           a1=m_age_1.pivot_table(values='weighted_average',index='title',columns='user_id').fillna(0)
           similarity=cosine_similarity(a1) 
    similar_items=sorted(list(enumerate(similarity[index])),key=lambda x:x[1],reverse=True)[1:6]

    for i in similar_items:
        l.append(a1.index[i[0]])
    return l   

In [73]:
a1

title,"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",2 Days in the Valley (1996),20 Dates (1998),"20,000 Leagues Under the Sea (1954)",...,Yellow Submarine (1968),Yojimbo (1961),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Your Friends and Neighbors (1998),Zero Effect (1998),eXistenZ (1999)
user_id,,,,,,,,,,,,,,,,,,,,,
18,0.0,0.0,0.000000,3.509804,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,4.686275,0.0,0.0,3.169935,0.0,0.000000,2.098039
152,0.0,0.0,4.764706,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
348,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
489,0.0,0.0,0.000000,4.176471,3.27451,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
579,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5756,0.0,0.0,0.000000,0.000000,0.00000,0.000000,2.660131,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5776,0.0,0.0,0.000000,0.000000,0.00000,4.921569,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5802,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000


In [74]:
recommend('101 Dalmatians (1996)')

TypeError: recommend() missing 2 required positional arguments: 'gender' and 'age_desc'

In [ ]:
df["age_desc"].unique()

array(['Under 18', '56+', '25-34', '50-55', '18-24', '45-49', '35-44'],
      dtype=object)

In [ ]:
import streamlit as st
st.header('Movie recommendation system')  #adding title
gender=df['gender'].unique()
age=['Under 18', '56+', '25-34', '50-55', '18-24', '45-49', '35-44']

#movie_list=final_data['original_title'].unique()
gender_selected=st.selectbox("select movie from dropdown",gender)
age=st.selectbox("select your age bracket",age) 
#selected_movie=st.selectbox("select movie from dropdown",movie_list)   # adding dropdown
if st.button('Show Recommendation'):
    recommended_movies=recommend(gender,age)
    recommended_movies

In [ ]:
from typing import List

In [ ]:
def get_user_id(gender,age):
    user_id=[]
    data___=pd.DataFrame()
    if gender=='M':
        if age==1:
            user_id.append(m_age_1['user_id'].unique())
            data___=m_age_1.copy()
        elif age==2:
            user_id.append(m_age_2['user_id'].unique())
            data___=m_age_2.copy()
        elif age==3:
            user_id.append(m_age_3['user_id'].unique())
            data___=m_age_3.copy()
        elif age==4:
            user_id.append(m_age_4['user_id'].unique())
            data___=m_age_4.copy()
        elif age==5:
            user_id.append(m_age_5['user_id'].unique())
            data___=m_age_5.copy()
        elif age==6:
            user_id.append(m_age_6['user_id'].unique())
            data___=m_age_6.copy()
        else:
            user_id.append(m_age_7['user_id'].unique())
            data___=m_age_6.copy()
    else:
        if age==1:
            user_id.append(f_age_1['user_id'].unique())
            data___=f_age_1.copy()
        elif age==2:
            user_id.append(f_age_2['user_id'].unique())
            data___=f_age_2.copy()
        elif age==3:
            user_id.append(f_age_3['user_id'].unique())
            data___=f_age_3.copy()
        elif age==4:
            user_id.append(f_age_4['user_id'].unique())
            data___=f_age_4.copy()
        elif age==5:
            user_id.append(f_age_5['user_id'].unique())
            data___=f_age_5.copy()
        elif age==6:
            user_id.append(f_age_6['user_id'].unique())
            data___=f_age_6.copy()
        else:
            user_id.append(f_age_7['user_id'].unique()) 
            data___=f_age_7.copy()

    return (user_id[0],data___)              


In [ ]:
m_age_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38405 entries, 6 to 991626
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rating            38405 non-null  int64  
 1   user_id           38405 non-null  int64  
 2   movie_id          38405 non-null  int64  
 3   title             38405 non-null  object 
 4   genres            38405 non-null  float64
 5   weighted_average  38405 non-null  float64
dtypes: float64(2), int64(3), object(1)
memory usage: 2.1+ MB


In [ ]:
def recommend_users(gender,age):
   l=[]
   mapping_dict_for_age={'Under 18':1, '56+':2, '25-34':3, '50-55':4, '18-24':5, '45-49':6, '35-44':7} 
   age=pd.Series(g).map(mapping_dict_for_age).values[0]
   user_id,reqd_data=get_user_id(gender,age)
   selected_user=st.selectbox("select your user_id",user_id) 
   pt=reqd_data.pivot_table(values='weighted_average',index='user_id',columns='title').fillna(0)
   similarity=cosine_similarity(pt)
   index=np.where(pt.index==selected_user)[0][0]
   similar_items=sorted(list(enumerate(similarity[selected_user])),key=lambda x:x[1],reverse=True)[1:6]

   for i in similar_items:
      l.append(pt.index[i[0]])

   return l,pt   

In [ ]:
def recommend(gender,age):
    l,pt=recommend_users(gender,age)
    movies_recommended=[]
    for i in l:
        ind_for_row=pt.index[i]
        ind_for_col=np.argmax(ii[ii.index==ii.index[4]],axis=1)
        movies_recommended.append(ii.columns[672])
    return movies_recommended

In [ ]:
pt=m_age_1.pivot_table(values='weighted_average',index='user_id',columns='title').fillna(0)

In [ ]:
pt

title,"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",2 Days in the Valley (1996),20 Dates (1998),"20,000 Leagues Under the Sea (1954)",...,Yellow Submarine (1968),Yojimbo (1961),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Your Friends and Neighbors (1998),Zero Effect (1998),eXistenZ (1999)
user_id,,,,,,,,,,,,,,,,,,,,,
18,0.0,0.0,0.000000,3.509804,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,4.686275,0.0,0.0,3.169935,0.0,0.000000,2.098039
152,0.0,0.0,4.764706,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
348,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
489,0.0,0.0,0.000000,4.176471,3.27451,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
579,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5756,0.0,0.0,0.000000,0.000000,0.00000,0.000000,2.660131,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5776,0.0,0.0,0.000000,0.000000,0.00000,4.921569,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5802,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000


In [ ]:
p

title,user_id,"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",2 Days in the Valley (1996),20 Dates (1998),...,Yellow Submarine (1968),Yojimbo (1961),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Your Friends and Neighbors (1998),Zero Effect (1998),eXistenZ (1999)
0,18,0.0,0.0,0.000000,3.509804,0.00000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,4.686275,0.0,0.0,3.169935,0.0,0.000000,2.098039
1,152,0.0,0.0,4.764706,0.000000,0.00000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
2,348,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
3,489,0.0,0.0,0.000000,4.176471,3.27451,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
4,579,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,5756,0.0,0.0,0.000000,0.000000,0.00000,0.000000,2.660131,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
140,5776,0.0,0.0,0.000000,0.000000,0.00000,4.921569,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
141,5802,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
142,5952,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,4.882353,4.098039


In [ ]:
np.nop.iloc[0].values[1:]

array([0.        , 0.        , 0.        , ..., 0.        , 0.        ,
       2.09803922])

In [ ]:
from numpy.linalg import norm

In [ ]:
p=pd.DataFrame(pt)
p=p.reset_index()
cos_simi=[]
ind=p.index[p['user_id']==18][0]
lst=p[p["user_id"]==18].values[0][1:]
for i in p.index:
    row_1=p.iloc[i].values[1:]
    cos_simi.append(np.dot(row_1,lst)/(norm(row_1))*norm(lst))
similar_items=sorted(list(enumerate(cos_simi)),key=lambda x:x[1],reverse=True)[1:6]     

In [ ]:
similar_items

[(100, 1512.3091532854066),
 (2, 1508.0079074065052),
 (104, 1358.971780050534),
 (27, 1349.0518732662324),
 (79, 1339.2666727539154)]

In [ ]:
similar_user_ind=[]
for i in similar_items:
    similar_user_ind.append(i[0])

In [ ]:
similar_user_ind

[100, 2, 104, 27, 79]

In [ ]:
pt

title,"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",2 Days in the Valley (1996),20 Dates (1998),"20,000 Leagues Under the Sea (1954)",...,Yellow Submarine (1968),Yojimbo (1961),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Your Friends and Neighbors (1998),Zero Effect (1998),eXistenZ (1999)
user_id,,,,,,,,,,,,,,,,,,,,,
18,0.0,0.0,0.000000,3.509804,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,4.686275,0.0,0.0,3.169935,0.0,0.000000,2.098039
152,0.0,0.0,4.764706,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
348,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
489,0.0,0.0,0.000000,4.176471,3.27451,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
579,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5756,0.0,0.0,0.000000,0.000000,0.00000,0.000000,2.660131,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5776,0.0,0.0,0.000000,0.000000,0.00000,4.921569,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5802,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000


In [ ]:
dd={}
for i in list(enumerate(pt.index)):
    dd[i[1]]=i[0]

In [ ]:
similar_user_ind

[100, 2, 104, 27, 79]

In [ ]:
dd[2]

KeyError: 2

In [ ]:
new_l=[]
for i in similar_user_ind:
    new_l.append(dd[i]) 

{18: 0,
 152: 1,
 348: 2,
 489: 3,
 579: 4,
 718: 5,
 736: 6,
 744: 7,
 752: 8,
 761: 9,
 774: 10,
 828: 11,
 865: 12,
 870: 13,
 874: 14,
 883: 15,
 905: 16,
 925: 17,
 939: 18,
 959: 19,
 978: 20,
 996: 21,
 1053: 22,
 1055: 23,
 1083: 24,
 1095: 25,
 1162: 26,
 1181: 27,
 1197: 28,
 1199: 29,
 1219: 30,
 1240: 31,
 1262: 32,
 1279: 33,
 1295: 34,
 1299: 35,
 1307: 36,
 1326: 37,
 1352: 38,
 1365: 39,
 1371: 40,
 1379: 41,
 1424: 42,
 1457: 43,
 1467: 44,
 1477: 45,
 1498: 46,
 1508: 47,
 1572: 48,
 1672: 49,
 1788: 50,
 1814: 51,
 1817: 52,
 1856: 53,
 1877: 54,
 1909: 55,
 1918: 56,
 1927: 57,
 1934: 58,
 1947: 59,
 1954: 60,
 1977: 61,
 1981: 62,
 2002: 63,
 2023: 64,
 2041: 65,
 2046: 66,
 2051: 67,
 2059: 68,
 2078: 69,
 2081: 70,
 2107: 71,
 2119: 72,
 2165: 73,
 2167: 74,
 2168: 75,
 2175: 76,
 2188: 77,
 2192: 78,
 2194: 79,
 2198: 80,
 2216: 81,
 2234: 82,
 2235: 83,
 2243: 84,
 2244: 85,
 2287: 86,
 2466: 87,
 2686: 88,
 2805: 89,
 2852: 90,
 2909: 91,
 3027: 92,
 3045: 93,

In [ ]:
similar_user=[]
for i in similar_user_ind:
    similar_user.append(p['user_id'][i])

In [ ]:
similar_user

[3593, 348, 4005, 1181, 2194]

In [ ]:
p["user_id"][2]

348

In [ ]:
p["user_id"][100]

3593

In [ ]:
p.iloc[]

title
user_id                              3593.0
'burbs, The (1989)                      0.0
...And Justice for All (1979)           0.0
10 Things I Hate About You (1999)       0.0
101 Dalmatians (1961)                   0.0
                                      ...  
Young Guns II (1990)                    0.0
Young Sherlock Holmes (1985)            0.0
Your Friends and Neighbors (1998)       0.0
Zero Effect (1998)                      0.0
eXistenZ (1999)                         0.0
Name: 100, Length: 1834, dtype: float64

In [ ]:
for i in sorted(list(enumerate(similarity[ind])),key=lambda x:x[1],reverse=True)[1:6]:
    print(p.index[i[0]])

100
2
104
27
79


In [ ]:
pt=m_age_1.pivot_table(values='weighted_average',index='user_id',columns='title').fillna(0)
similarity=cosine_similarity(pt)
user=232
ll=[]
index=pt.loc[user]
similar_items=sorted(list(enumerate(similarity[index])),key=lambda x:x[1],reverse=True)[1:6]
for i in similar_items:
    ll.append(i)

KeyError: 232

In [ ]:
pt.loc[232]

KeyError: 232

In [ ]:
a=pt.loc[152]
a.values

(1833,)

In [ ]:
similarity.shape

(144, 144)

In [ ]:
for i,j in list(enumerate(similarity)):
    if a.values==j:
        print(i)
        break

ValueError: operands could not be broadcast together with shapes (1833,) (144,) 

In [ ]:
pt

title,"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",2 Days in the Valley (1996),20 Dates (1998),"20,000 Leagues Under the Sea (1954)",...,Yellow Submarine (1968),Yojimbo (1961),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Your Friends and Neighbors (1998),Zero Effect (1998),eXistenZ (1999)
user_id,,,,,,,,,,,,,,,,,,,,,
18,0.0,0.0,0.000000,3.509804,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,4.686275,0.0,0.0,3.169935,0.0,0.000000,2.098039
152,0.0,0.0,4.764706,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
348,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
489,0.0,0.0,0.000000,4.176471,3.27451,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
579,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5756,0.0,0.0,0.000000,0.000000,0.00000,0.000000,2.660131,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5776,0.0,0.0,0.000000,0.000000,0.00000,4.921569,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5802,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000


In [ ]:
similarity.shape

(144, 144)

In [ ]:
pt.index==user
np.where(index)

(array([], dtype=int64),)

In [ ]:
m_age_1.drop_duplicates()

,rating,user_id,movie_id,title,genres,weighted_average
6,5,18,1192,One Flew Over the Cuckoo's Nest (1975),4.764706,4.921569
103,4,348,1192,One Flew Over the Cuckoo's Nest (1975),4.764706,4.254902
203,4,718,1192,One Flew Over the Cuckoo's Nest (1975),4.764706,4.254902
234,4,828,1192,One Flew Over the Cuckoo's Nest (1975),4.764706,4.254902
268,4,959,1192,One Flew Over the Cuckoo's Nest (1975),4.764706,4.254902
...,...,...,...,...,...,...
989581,1,5557,2094,"Shaggy D.A., The (1976)",2.647059,1.549020
989581,1,5557,2094,"Shaggy D.A., The (1976)",5.000000,2.333333
990704,5,5254,1361,"Garden of Finzi-Contini, The (Giardino dei Fin...",4.764706,4.921569
991626,4,5254,3444,Eyes of Laura Mars (1978),2.176471,3.392157


In [ ]:
pt=m_age_1.pivot_table(values='weighted_average',index='user_id',columns='title').fillna(0)
similarity=cosine_similarity(pt)
user_id=348
index=pt.loc[user_id]
index

pandas.core.series.Series

In [ ]:
pd.DataFrame(similarity)

,0,1,2,3,4,5,6,7,8,9,...,134,135,136,137,138,139,140,141,142,143
0,1.000000,0.088441,0.426138,0.276566,0.069672,0.172090,0.290698,0.056303,0.281165,0.274832,...,0.335835,0.181539,0.192444,0.234506,0.357700,0.280968,0.211941,0.181867,0.287562,0.224881
1,0.088441,1.000000,0.114605,0.000000,0.000000,0.000000,0.104743,0.000000,0.180729,0.000000,...,0.017056,0.040140,0.000000,0.031799,0.084386,0.057324,0.068064,0.036287,0.082825,0.044093
2,0.426138,0.114605,1.000000,0.135995,0.148347,0.197303,0.333224,0.106396,0.314854,0.240217,...,0.191988,0.192564,0.048544,0.277738,0.337245,0.247431,0.249987,0.177057,0.186766,0.236542
3,0.276566,0.000000,0.135995,1.000000,0.050220,0.056500,0.210312,0.021805,0.197656,0.393702,...,0.443666,0.128494,0.307866,0.090612,0.156641,0.104854,0.080775,0.082392,0.237357,0.205246
4,0.069672,0.000000,0.148347,0.050220,1.000000,0.033561,0.075465,0.000000,0.000000,0.134610,...,0.050470,0.000000,0.028183,0.020412,0.113563,0.158645,0.184438,0.052899,0.056209,0.092031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,0.280968,0.057324,0.247431,0.104854,0.158645,0.120665,0.330762,0.085870,0.108100,0.147826,...,0.185857,0.129605,0.062273,0.084550,0.357784,1.000000,0.540656,0.130237,0.273444,0.240316
140,0.211941,0.068064,0.249987,0.080775,0.184438,0.172333,0.331975,0.061206,0.074225,0.110125,...,0.119128,0.174503,0.041909,0.049797,0.279111,0.540656,1.000000,0.109155,0.173351,0.149436
141,0.181867,0.036287,0.177057,0.082392,0.052899,0.132862,0.136979,0.035903,0.086153,0.105931,...,0.177075,0.064956,0.126108,0.147934,0.264480,0.130237,0.109155,1.000000,0.225570,0.067148
142,0.287562,0.082825,0.186766,0.237357,0.056209,0.120452,0.296793,0.023766,0.169693,0.281192,...,0.351941,0.083711,0.115900,0.171610,0.330959,0.273444,0.173351,0.225570,1.000000,0.278525


In [ ]:
pd.DataFrame(similarity).loc[user_id]

KeyError: 348

In [ ]:
#recommend users
l=[]
mapping_dict_for_age={'Under 18':1, '56+':2, '25-34':3, '50-55':4, '18-24':5, '45-49':6, '35-44':7} 
age=pd.Series(age).map(mapping_dict_for_age).values[0]
pt=m_age_1.pivot_table(values='weighted_average',index='user_id',columns='title').fillna(0)
similarity=cosine_similarity(pt)
user_id=348
index=pt.loc[user_id]
similar_items=sorted(list(enumerate(similarity[index])),key=lambda x:x[1],reverse=True)[1:6]

for i in similar_items:
    l.append(pt.index[i[0]])


In [ ]:
pt

title,"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",2 Days in the Valley (1996),20 Dates (1998),"20,000 Leagues Under the Sea (1954)",...,Yellow Submarine (1968),Yojimbo (1961),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Your Friends and Neighbors (1998),Zero Effect (1998),eXistenZ (1999)
user_id,,,,,,,,,,,,,,,,,,,,,
18,0.0,0.0,0.000000,3.509804,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,4.686275,0.0,0.0,3.169935,0.0,0.000000,2.098039
152,0.0,0.0,4.764706,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
348,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
489,0.0,0.0,0.000000,4.176471,3.27451,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
579,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5756,0.0,0.0,0.000000,0.000000,0.00000,0.000000,2.660131,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5776,0.0,0.0,0.000000,0.000000,0.00000,4.921569,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5802,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000


In [ ]:
pt=m_age_1.pivot_table(values='weighted_average',index='user_id',columns='title').fillna(0)
pt

title,"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",2 Days in the Valley (1996),20 Dates (1998),"20,000 Leagues Under the Sea (1954)",...,Yellow Submarine (1968),Yojimbo (1961),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Your Friends and Neighbors (1998),Zero Effect (1998),eXistenZ (1999)
user_id,,,,,,,,,,,,,,,,,,,,,
18,0.0,0.0,0.000000,3.509804,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,4.686275,0.0,0.0,3.169935,0.0,0.000000,2.098039
152,0.0,0.0,4.764706,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
348,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
489,0.0,0.0,0.000000,4.176471,3.27451,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
579,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5756,0.0,0.0,0.000000,0.000000,0.00000,0.000000,2.660131,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5776,0.0,0.0,0.000000,0.000000,0.00000,4.921569,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5802,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000


In [ ]:
ind=p.index[p['user_id']==348][0]
ind

2

In [ ]:
pt=m_age_1.pivot_table(values='weighted_average',index='user_id',columns='title').fillna(0)
pt1=pt
l=[]
user_id=348
p=pd.DataFrame(pt)
p=p.reset_index()
cos_simi=[]
ind=p.index[p['user_id']==user_id][0]
lst=p[p["user_id"]==user_id].values[0][1:]
for i in p.index:
    row_1=p.iloc[i].values[1:]
    cos_simi.append(np.dot(row_1,lst)/(norm(row_1))*norm(lst))
similar_items=sorted(list(enumerate(cos_simi)),key=lambda x:x[1],reverse=True)[1:6]    

for i in similar_items:
    l.append(pt.index[i[0]])


In [ ]:
pt

title,"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",2 Days in the Valley (1996),20 Dates (1998),"20,000 Leagues Under the Sea (1954)",...,Yellow Submarine (1968),Yojimbo (1961),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Your Friends and Neighbors (1998),Zero Effect (1998),eXistenZ (1999)
user_id,,,,,,,,,,,,,,,,,,,,,
18,0.0,0.0,0.000000,3.509804,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,4.686275,0.0,0.0,3.169935,0.0,0.000000,2.098039
152,0.0,0.0,4.764706,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
348,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
489,0.0,0.0,0.000000,4.176471,3.27451,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
579,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5756,0.0,0.0,0.000000,0.000000,0.00000,0.000000,2.660131,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5776,0.0,0.0,0.000000,0.000000,0.00000,4.921569,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5802,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000


In [ ]:
list(pt.index).index(152)

1

In [ ]:
p

title,user_id,"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",2 Days in the Valley (1996),20 Dates (1998),...,Yellow Submarine (1968),Yojimbo (1961),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Your Friends and Neighbors (1998),Zero Effect (1998),eXistenZ (1999)
0,18,0.0,0.0,0.000000,3.509804,0.00000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,4.686275,0.0,0.0,3.169935,0.0,0.000000,2.098039
1,152,0.0,0.0,4.764706,0.000000,0.00000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
2,348,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
3,489,0.0,0.0,0.000000,4.176471,3.27451,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
4,579,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,5756,0.0,0.0,0.000000,0.000000,0.00000,0.000000,2.660131,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
140,5776,0.0,0.0,0.000000,0.000000,0.00000,4.921569,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
141,5802,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
142,5952,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,4.882353,4.098039


In [ ]:
p.index[p['user_id']==152][0]

1

In [ ]:
aa=p[p['user_id']==18].values[0][1:]
aa

array([0.        , 0.        , 0.        , ..., 0.        , 0.        ,
       2.09803922])

In [ ]:
p.columns.tolist()[224]

'Blue Angel, The (Blaue Engel, Der) (1930)'

In [ ]:
np.argmax(aa,axis=0)

224

In [ ]:
pt.index[0]

18

In [ ]:
pt1.index[0]

18

In [ ]:
pt

title,"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",2 Days in the Valley (1996),20 Dates (1998),"20,000 Leagues Under the Sea (1954)",...,Yellow Submarine (1968),Yojimbo (1961),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Your Friends and Neighbors (1998),Zero Effect (1998),eXistenZ (1999)
user_id,,,,,,,,,,,,,,,,,,,,,
18,0.0,0.0,0.000000,3.509804,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,4.686275,0.0,0.0,3.169935,0.0,0.000000,2.098039
152,0.0,0.0,4.764706,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
348,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
489,0.0,0.0,0.000000,4.176471,3.27451,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
579,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5756,0.0,0.0,0.000000,0.000000,0.00000,0.000000,2.660131,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5776,0.0,0.0,0.000000,0.000000,0.00000,4.921569,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5802,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000


In [ ]:
pt1

title,"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",2 Days in the Valley (1996),20 Dates (1998),"20,000 Leagues Under the Sea (1954)",...,Yellow Submarine (1968),Yojimbo (1961),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Your Friends and Neighbors (1998),Zero Effect (1998),eXistenZ (1999)
user_id,,,,,,,,,,,,,,,,,,,,,
18,0.0,0.0,0.000000,3.509804,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,4.686275,0.0,0.0,3.169935,0.0,0.000000,2.098039
152,0.0,0.0,4.764706,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
348,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
489,0.0,0.0,0.000000,4.176471,3.27451,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
579,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5756,0.0,0.0,0.000000,0.000000,0.00000,0.000000,2.660131,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5776,0.0,0.0,0.000000,0.000000,0.00000,4.921569,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5802,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000


In [ ]:
list(enumerate(pt.index[]))

[(0, 18),
 (1, 152),
 (2, 348),
 (3, 489),
 (4, 579),
 (5, 718),
 (6, 736),
 (7, 744),
 (8, 752),
 (9, 761),
 (10, 774),
 (11, 828),
 (12, 865),
 (13, 870),
 (14, 874),
 (15, 883),
 (16, 905),
 (17, 925),
 (18, 939),
 (19, 959),
 (20, 978),
 (21, 996),
 (22, 1053),
 (23, 1055),
 (24, 1083),
 (25, 1095),
 (26, 1162),
 (27, 1181),
 (28, 1197),
 (29, 1199),
 (30, 1219),
 (31, 1240),
 (32, 1262),
 (33, 1279),
 (34, 1295),
 (35, 1299),
 (36, 1307),
 (37, 1326),
 (38, 1352),
 (39, 1365),
 (40, 1371),
 (41, 1379),
 (42, 1424),
 (43, 1457),
 (44, 1467),
 (45, 1477),
 (46, 1498),
 (47, 1508),
 (48, 1572),
 (49, 1672),
 (50, 1788),
 (51, 1814),
 (52, 1817),
 (53, 1856),
 (54, 1877),
 (55, 1909),
 (56, 1918),
 (57, 1927),
 (58, 1934),
 (59, 1947),
 (60, 1954),
 (61, 1977),
 (62, 1981),
 (63, 2002),
 (64, 2023),
 (65, 2041),
 (66, 2046),
 (67, 2051),
 (68, 2059),
 (69, 2078),
 (70, 2081),
 (71, 2107),
 (72, 2119),
 (73, 2165),
 (74, 2167),
 (75, 2168),
 (76, 2175),
 (77, 2188),
 (78, 2192),
 (79, 

In [ ]:
pt.index

Index([  18,  152,  348,  489,  579,  718,  736,  744,  752,  761,
       ...
       5557, 5636, 5641, 5661, 5715, 5756, 5776, 5802, 5952, 5972],
      dtype='int64', name='user_id', length=144)

In [ ]:
l

[5554, 18, 1083, 1467, 5073]

In [ ]:
similar_items


[(133, 1659.321780609496),
 (0, 1533.4042669876),
 (24, 1530.928333394198),
 (44, 1506.8797732044625),
 (123, 1504.0696847272893)]

In [85]:
user_id=[]
user_id.append(m_age_6['user_id'].unique())
user_id[0]

43

In [86]:
p

NameError: name 'p' is not defined

In [114]:
a=[1,3,4]
b=[1,1,1]
c=[i+j for i in a for j in b]
list(set(c))

[2, 4, 5]

In [119]:
pt=m_age_1.pivot_table(values='weighted_average',index='user_id',columns='title').fillna(0)

In [121]:
df_female['age_desc'].unique()

array([1, 2, 3, 4, 5, 6, 7], dtype=int64)

In [103]:
pt

title,"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",2 Days in the Valley (1996),20 Dates (1998),"20,000 Leagues Under the Sea (1954)",...,Yellow Submarine (1968),Yojimbo (1961),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Your Friends and Neighbors (1998),Zero Effect (1998),eXistenZ (1999)
user_id,,,,,,,,,,,,,,,,,,,,,
18,0.0,0.0,0.000000,3.509804,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,4.686275,0.0,0.0,3.169935,0.0,0.000000,2.098039
152,0.0,0.0,4.764706,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
348,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
489,0.0,0.0,0.000000,4.176471,3.27451,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
579,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5756,0.0,0.0,0.000000,0.000000,0.00000,0.000000,2.660131,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5776,0.0,0.0,0.000000,0.000000,0.00000,4.921569,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5802,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000


In [104]:

p=pd.DataFrame(pt)

In [105]:
len(p.columns.tolist())

1833

In [106]:
len(list(set(p.columns)))

1833